# Data Analysis South German Credit Data

Data and code from
https://www.kaggle.com/code/vasukhajuria/south-german-credit-risk-analysis/notebook

Extensive description of data set and its creation: http://www1.beuth-hochschule.de/FB_II/reports/Report-2019-004.pdf

The data are a stratified sample of 1000 credits (300 bad ones and 700 good ones) from the years 1973 to 1975 from a large regional bank in southern Germany, which had about 500 branches, among them both urban and rural ones. Bad credits are heavily oversampled, in order to acquire sufficient information for discriminating them from good ones; the sources report that
the actual prevalence of bad credits is around 5%. In a Bayesian context, 5% might be used as a prior probability for a credit being bad. 

As suggested with the Statlog German credit data, one might consider misclassification cost, and it has been suggested to allocate the cost for misclassifying a bad risk as good to be five times as high than the cost for misclassifying a good risk as bad.

### Variable description

First name original german version, then English translation.

1. laufkont -- status
:  status of the debtor’s checking account with the bank (categorical)

2. laufzeit -- duration
:  credit duration in months (quantitative)

3. moral -- credit_history
:  history of compliance with previous or concurrent credit contracts (categorical)

4. verw -- purpose
:  purpose for which the credit is needed (categorical)

5. hoehe -- amount
:  credit amount in DM (quantitative; result of monotonic transformation; actual data and type of transformation unknown)

6. sparkont -- savings
:  debtor’s savings (categorical)

7. beszeit -- employment_duration
:  duration of debtor’s employment with current employer (ordinal; discretized quantitative)

8. rate -- installment_rate
:  credit installments as a percentage of debtor’s disposable income (ordinal; discretized quantitative)

9. famges -- personal_status_sex
:  combined information on sex and marital status; categorical; sex cannot be recovered from the
variable, because male singles and female non-singles are coded with the same code (2); female widows
cannot be easily classified, because the code table does not list them in any of the female categories

10. buerge -- other_debtors
:  Is there another debtor or a guarantor for the credit? (categorical)

11. wohnzeit -- present_residence
:  length of time (in years) the debtor lives in the present residence (ordinal; discretized quantitative)

12. verm -- property
:  the debtor’s most valuable property, i.e. the highest possible code is used. Code 2 is used, if
codes 3 or 4 are not applicable and there is a car or any other relevant property that does not fall under
variable sparkont. (ordinal)

13. alter -- age
:  age in years (quantitative)

14. weitkred -- other_installment_plans
:  installment plans from providers other than the credit-giving bank (categorical)

15. wohn -- housing
:  type of housing the debtor lives in (categorical)

16. bishkred -- number_credits
:  number of credits including the current one the debtor has (or had) at this bank (ordinal,
discretized quantitative); contrary to Fahrmeir and Hamerle’s (1984) statement, the original data values
are not available.

17. beruf -- job
:  quality of debtor’s job (ordinal)

18. pers -- people_liable
:  number of persons who financially depend on the debtor (i.e., are entitled to maintenance)
(binary, discretized quantitative)

19. telef -- telephone
:  Is there a telephone landline registered on the debtor’s name? (binary; remember that the data
are from the 1970s)

20. gastarb -- foreign_worker
:  Is the debtor a foreign worker? (binary)

21. kredit -- credit_risk
:  Has the credit contract been complied with (good) or not (bad) ? (binary)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
from scipy.stats import norm,skew

from subprocess import check_output

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('SouthGermanCredit_clean.csv')

In [ ]:
pd.set_option('display.max_columns', 25)
df.head()

In [ ]:
df.info()

## EDA

In [ ]:
sns.countplot(df['credit_risk'])

70% of the total records have good credit risk and 30% of records have bad credit risk.¶


### Categorical data

In [ ]:
categorical_columns = ['status','credit_history','purpose','savings','employment_duration','installment_rate','personal_status_sex','other_debtors','present_residence','property','other_installment_plans','housing','number_credits','job','people_liable','telephone','foreign_worker']

In [ ]:
# status
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='status',data=df, order=df['status'].value_counts().index)

- 39.4% of records have 200 DM or more balance in checking account
- 27.4% of records have no checking account.
- 26.9% of records have negative balance.
- Only 6.3% of records have balance greater than 0 and less than 200 DM in checing account

In [ ]:
# credit_history
fig_dims = (20, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='credit_history', order=df['credit_history'].value_counts().index, data=df)

In [ ]:
# purpose
fig_dims = (17, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='purpose', order=df['purpose'].value_counts().index, data=df)

In [ ]:
# savings
fig_dims = (15, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='savings', order=df['savings'].value_counts().index,data=df)

- More than 60% of records have either no savings accounts or it's unknown.


In [ ]:
# employment_duration
fig_dims = (10, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='employment_duration', order=df['employment_duration'].value_counts().index,data=df)

In [ ]:
# installment_rate of their disposable income
fig_dims = (8, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='installment_rate', order=df['installment_rate'].value_counts().index, data=df)

- 47.6% of people have installment rate less than 20% of their disposable (net) income.
- 23.1% of people have installment rate between 25 to 35% of their disposable income.
- 15.7% of people have installment rate between 20 to 25% of their disposable income.
- 13.6% of records have installment rate greater than 35% of their disposable income. It means high loans are taken by only 13.6% of the people.

In [ ]:
# personal_status_sex
fig_dims = (15, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='personal_status_sex', order=df['personal_status_sex'].value_counts().index, data=df)

- 54.8% of credits are taken by males who are either married or widowed.
- 31% of credits are taken by females who are married or males who are single.
- 9.2% of credits are taken by females who are single.
- 5% of credits are taken by males who are divorced/separated.

In [ ]:
# other_debtors
fig_dims = (10, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='other_debtors', order=df['other_debtors'].value_counts().index, data=df)

In [ ]:
# present_residence
fig_dims = (10, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='present_residence', order=df['present_residence'].value_counts().index, data=df)

In [ ]:
# property
fig_dims = (15, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='property', order=df['property'].value_counts().index, data=df)

- 33.2% of credits are given to people who have building society, savings or life insurance as their most valued property.
- 28.2% of credits are given to people who either don't have any property or it's unknown. This is critical as it can lead to bad credit risks.
- 23.2 % of credits are granted to people whose most valued property is car or any other thing.
- 15.4 % of credits are granted to people whose most valued property is real estate.

In [ ]:
# other_installment_plans
fig_dims = (8,5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='other_installment_plans', order=df['other_installment_plans'].value_counts().index, data=df)

- 81.4% of records have no other installment plans.
- 13.9% of records have other installment plans going on from banks other than the credit-giving bank.
- 4.7% of records have other installment plans from stores.

In [ ]:
# housing
fig_dims = (8, 5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='housing', order=df['housing'].value_counts().index, data=df)

In [ ]:
# number_credits at this bank
fig_dims = (8, 5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='number_credits', order=df['number_credits'].value_counts().index, data=df)

In [ ]:
# job
fig_dims = (18, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='job', order=df['job'].value_counts().index, data=df)

In [ ]:
# people_liable - number of people who are financially dependent on debtor
fig_dims = (8, 5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='people_liable', order=df['people_liable'].value_counts().index, data=df)

In [ ]:
# telephone 
# Is there a telephone landline registered on the debtor’s name? (binary; remember that the data are from the 1970s)
fig_dims = (8, 5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='telephone', order=df['telephone'].value_counts().index, data=df)

In [ ]:
# foreign_worker
fig_dims = (8, 5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x='foreign_worker', order=df['foreign_worker'].value_counts().index, data=df)

### Numerical data
As can be seen below, all of the numerical variables have some outliers and are right skewed. In order to get better results for the culster analysis, we use a power transformation. 

In [ ]:
sns.distplot(df['duration'])
print('Skewness of duration:',df['duration'].skew())

In [ ]:
sns.distplot(df['duration'])
print('Skewness of duration:',df['duration'].skew())
sns.boxplot(df['duration'])

Only few outliers in 'duration' column.
Data is right skewed.

In [ ]:
sns.distplot(df['amount'])
print('Skewness of duration:',df['amount'].skew())

In [ ]:
sns.distplot(df['amount'])
print('Skewness of duration:',df['amount'].skew())
sns.boxplot(df['amount'])

Lot of outliers in 'amount' column.
Data is highly right skewed.

In [ ]:
sns.distplot(df['age'])
print('Skewness of age:',df['age'].skew())

In [ ]:
sns.distplot(df['age'])
print('Skewness of age:',df['age'].skew())
sns.boxplot(df['age'])

Some outliers in 'age' column.
Data is slightly right skewed.

### Bivariante analysis

In [ ]:
corr = df.corr()
sns.heatmap(corr, cmap="Greens",annot=True)

In [ ]:
sns.pairplot(df, hue = 'credit_risk')

#### Bivariate analysis could be continued for a good understanding of the data

### Outlier Treatment

In [ ]:
# Power Transformer
from sklearn.preprocessing import PowerTransformer

In [ ]:
numerical_columns = ['duration','amount','age']

In [ ]:
df_continuous = df[numerical_columns]
df_continuous.head()

In [ ]:
pt = PowerTransformer(method='box-cox')
power_transformed_array = pt.fit_transform(df_continuous)

In [ ]:
ptdf_cont = pd.DataFrame(data = power_transformed_array, columns = df_continuous.columns)
ptdf_cont.head()

In [ ]:
for i in ptdf_cont.columns:
    sns.boxplot(ptdf_cont[i])
    print("skewness of ",i,":",ptdf_cont[i].skew())
    plt.show()  

In [ ]:
# new dataframed with power-transformed variables
ptdf = df.copy()

for col in ptdf_cont.columns:
    ptdf[col] = ptdf_cont[col]

In [ ]:
sns.heatmap(ptdf[numerical_columns].corr(),annot=True)

In [ ]:
# A variance inflation factor (VIF) is a measure of the amount of 
# multicollinearity in regression analysis
# As a rule of thumb, a VIF of three or below is not a cause for concern
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

vf = [vif(ptdf[numerical_columns].values, i ) for i in range(ptdf[numerical_columns].shape[1])]
pd.DataFrame(vf, index=numerical_columns, columns= ["vif"])

In [ ]:
ptdf.head()

## K Means Cluster Creation

In [ ]:
# for running KPrototypes which is a variant of k-means that can handle categorical and numerical variables

In [ ]:
pip install kmodes 

In [ ]:
# Import the necessary libraries
import pandas as pd
from kmodes.kprototypes import KPrototypes

# Separate the numerical and categorical variables into two separate dataframes
# num_df = df.select_dtypes(include=['int', 'float']) # Numerical variables
# cat_df = df.select_dtypes(include=['object']) # Categorical variables

#### Preprocess the data if necessary
One Hot Encoding: For categorical variables where no ordinal relationship exists, integer encoding (i.e. transforming each category of a variable into a number)  is not enough.

In fact, using this encoding and allowing the model to assume a natural ordering between categories may result in poor performance or unexpected results (predictions halfway between categories).

In this case, a one-hot encoding can be applied to the integer representation. This is where the integer encoded variable is removed and a new binary variable is added for each unique integer value.

In [ ]:
ptdf.info()

In [ ]:
ptdf_encoded = pd.get_dummies(ptdf, columns=['status', 'credit_history', 'purpose', 'savings', 'employment_duration',
                                        'installment_rate', 'personal_status_sex', 'other_debtors', 'present_residence',
                                        'property', 'other_installment_plans', 'housing', 'number_credits', 'job',
                                        'people_liable', 'telephone', 'foreign_worker', 'credit_risk'])

In [ ]:
# in new data frame df_encoded categorical varialbes are replaced with dummy variables
ptdf_encoded.head(5)

In [ ]:
# defining variables that are categorical indicators
num_cols = ['duration', 'amount', 'age']
cat_cols = [col for col in ptdf_encoded.columns if col not in num_cols]

cat_indices = [ptdf_encoded.columns.get_loc(col) for col in cat_cols]

In [ ]:
# Run cluster analysis 
cost = []
for num_clusters in range(1, 10):
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao', n_init=10, max_iter=20, random_state=0)
    clusters = kproto.fit_predict(ptdf_encoded.values, categorical=cat_indices)
    try:
        cost.append(kproto.cost_)
    except:
        print(f"Error computing cost for num_clusters = {num_clusters}")
    print(f"num_clusters = {num_clusters}, cost = {kproto.cost_}")


In [ ]:
# Plot the elbow curve to visually identify the optimal number of clusters
plt.plot(range(1, 10), cost)
#alternatively if line before does not work: plt.plot(range(1, len(cost)+1), cost)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Cost')
plt.show()

In [ ]:
# Based on the elbow curve, determine the optimal number of clusters and fit the k-prototypes model
kproto = KPrototypes(n_clusters=3, init='Cao', n_init=10, max_iter=20, random_state=0)
clusters = kproto.fit_predict(ptdf_encoded.values, categorical=cat_indices)

In [ ]:
# Add the cluster labels to the dataframe
ptdf_encoded['Cluster'] = clusters

# Add the cluster names to the original data set
df['Cluster'] = clusters

In [ ]:
ptdf_encoded.head()

In [ ]:
df.head()

## EDA with clusters

#### Numerical variables

In [ ]:
# assume the numerical variables in df are stored in columns that have numerical data type
num_cols = df.select_dtypes(include=['int64', 'float64', 'uint16']).columns.tolist()

# create a new dataframe called df_num with numerical and 'Cluster' variables
df_num = df[num_cols]

df_num.head()

In [ ]:
df_num_mean = df_num.groupby('Cluster').mean()
df_num_mean 

In [ ]:
df_num_median = df_num.groupby('Cluster').median()
df_num_median

In [ ]:
# Save the DataFrame as an Excel file
filename = 'cluster_numerical_result.xlsx'  # Set the filename
df_num_median.to_excel(filename, index=True)

0: largest amount of credit, longest duration, only middle age

1: old, smaller amount and smaller duration

2: youngest, smaller amount and smaller duration

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# assume df contains numerical variables 'duration', 'amount', 'age', and a categorical variable 'Cluster'

# create a 3D scatter plot of 'duration', 'amount', and 'age' for each value of 'Cluster'
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.view_init(elev=30, azim=115  )  # add rotation
colors = ['r', 'g', 'b']

for i, c in enumerate(df['Cluster'].unique()):
    x = df.loc[df['Cluster'] == c, 'duration']
    y = df.loc[df['Cluster'] == c, 'amount']
    z = df.loc[df['Cluster'] == c, 'age']
    ax.scatter(x, y, z, c=colors[i], alpha=0.2, label='Cluster {}'.format(c))

ax.set_xlabel('Duration')
ax.set_ylabel('Amount')
ax.set_zlabel('Age')
ax.legend()

plt.show()

2: closest toghether

0: farthest apart

In [ ]:
sns.pairplot(df, hue = 'Cluster', palette=['b', 'g', 'r'])

#### Categorical variables

In [ ]:
# assume the categorical variables in df are stored in columns that have categorical data type
cat_cols = df.select_dtypes(include=['object', 'uint16']).columns.tolist()

# create a new dataframe called df_num with numerical and 'Cluster' variables
df_cat = df[cat_cols]

df_cat.head()

In [ ]:
# Group by 'Cluster' and show percentage of each category rounded to 0 decimal places
for col in df_cat.columns:
    print(df_cat.groupby('Cluster')[col].value_counts(normalize=True).mul(100).round(0))



In [ ]:
!pip install xlwt

In [ ]:
# save as an xls table

result = pd.DataFrame(columns=['Cluster', 'Column', 'Category', 'Percentage'])

# Add Cluster column to result DataFrame
#result['Cluster'] = df_cat['Cluster']

for col in df_cat.columns.drop('Cluster'):
    sub_df = df_cat.groupby('Cluster')[col].value_counts(normalize=True).mul(100).round(2).reset_index(name='Percentage')
    sub_df.columns = ['Cluster', 'Category', 'Percentage']
    sub_df['Column'] = col
    result = pd.concat([result, sub_df])

# Sort the result DataFrame by Cluster and Column
result = result.sort_values(['Cluster', 'Column'])

# Reset the index of the result DataFrame
result = result.reset_index(drop=True)

# save
result.to_excel('cluster_category_result.xls', index=False)